# LGBM + CatBoost + XGBoost 앙상블

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample_submission.csv
/kaggle/input/train.csv
/kaggle/input/test.csv


In [5]:
data_path = '/kaggle/input/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [6]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

In [7]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [8]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    all_data[col] = all_data[col].astype(str)

In [9]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[categorical_columns])

encoded_cat_matrix

<346418x205 sparse matrix of type '<class 'numpy.float64'>'
	with 16281646 stored elements in Compressed Sparse Row format>

In [10]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [11]:
all_columns = all_data.columns
all_columns

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '특정 시술 유형',
       '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수',
       '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처',
       '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자

In [12]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[numeric_columns]),
                               encoded_cat_matrix],
                              format='csr')

## 2. 데이터 나누기

In [13]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

## 3. ROC-AUC 정의

In [14]:
from sklearn.metrics import roc_auc_score

# LGBM 커스텀 평가 함수
def lgb_roc_auc(y_pred, dataset):
    y_true = dataset.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred), True  # (지표 이름, 값, 높은 값이 더 좋은지 여부)

In [15]:
# XGBoost 커스텀 평가 함수
def xgb_roc_auc(y_pred, dtrain):
    y_true = dtrain.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred)

## 4. 층화 K 폴드 설정

In [16]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

## 5. 모델 훈련 

### 5-1. LGBM

In [17]:
max_params_lgb = {
	'bagging_fraction': 0.5863913416418778,
	'feature_fraction': 0.9581372279421587,
	'lambda_l1': 0.3465698371871525,
	'lambda_l2': 1.839362361400739,
	'min_child_samples': 49,
	'min_child_weight': 41.152903682697854,
	'num_leaves': 31,
	'objective': 'binary',
	'learning_rate': 0.005,
	'bagging_freq': 1,
	'force_row_wise': True,
	'random_state': 1991
 }

In [18]:
import lightgbm as lgb

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds_lgb = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds_lgb = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params_lgb,     # 최적 하이퍼파라미터
                          train_set=dtrain,          # 훈련 데이터셋
                          num_boost_round=2500,      # 부스팅 반복 횟수
                          valid_sets=dvalid,         # 성능 평가용 검증 데이터셋
                          feval = lgb_roc_auc,
                          callbacks=[lgb.early_stopping(stopping_rounds=200)])
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds_lgb += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds_lgb[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(y_valid, oof_val_preds_lgb[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59605, number of negative: 171110
[LightGBM] [Info] Total Bins 883
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 187
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054567
[LightGBM] [Info] Start training from score -1.054567
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[2389]	valid_0's binary_logloss: 0.487666	valid_0's roc_auc: 0.740225
폴드 1 roc-auc : 0.7402252385799367

######################################## 폴드 2 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59606, number of negative: 171110
[LightGBM] [Info] Total Bins 884
[LightGBM] [Info] Number of data points in the train set: 230716, number of used features: 187
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> i

### 5-2. CatBoost

In [20]:
max_params_catboost = {
	'bagging_temperature': 0.9210971924216698,
	'border_count': 210,
	'depth': 7,
	'l2_leaf_reg': 8.360830470231202,
	'learning_rate': 0.07314376779212872,
	'loss_function': 'Logloss',
	'eval_metric': 'AUC',
	'iterations': 2500,
	'random_seed': 1991,
	'verbose': 200
 }

In [21]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# OOF 방식으로 모델 훈련, 검증, 예측
oof_val_preds_catboost = np.zeros(X.shape[0])
oof_test_preds_catboost = np.zeros(X_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    
    dtrain = Pool(X_train, y_train, cat_features=None)
    dvalid = Pool(X_valid, y_valid, cat_features=None)
    
    cat_model = CatBoostClassifier(**max_params_catboost)
    cat_model.fit(dtrain, 
                  eval_set=dvalid, 
                  early_stopping_rounds=200, 
                  verbose=500)
    
    # 테스트 데이터 예측
    oof_test_preds_catboost += cat_model.predict_proba(X_test)[:, 1] / folds.n_splits
    # 검증 데이터 예측
    oof_val_preds_catboost[valid_idx] += cat_model.predict_proba(X_valid)[:, 1]
    
    # 검증 데이터 ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, oof_val_preds_catboost[valid_idx])
    print(f'폴드 {idx+1} ROC-AUC : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
0:	test: 0.6993516	best: 0.6993516 (0)	total: 94.7ms	remaining: 3m 56s
500:	test: 0.7396828	best: 0.7399418 (461)	total: 19.8s	remaining: 1m 19s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7399417918
bestIteration = 461

Shrink model to first 462 iterations.
폴드 1 ROC-AUC : 0.7397682731744077

######################################## 폴드 2 / 폴드 10 ########################################
0:	test: 0.6327822	best: 0.6327822 (0)	total: 44.6ms	remaining: 1m 51s
500:	test: 0.7379100	best: 0.7380729 (434)	total: 19.5s	remaining: 1m 17s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7380728811
bestIteration = 434

Shrink model to first 435 iterations.
폴드 2 ROC-AUC : 0.7386987305558932

######################################## 폴드 3 / 폴드 10 ########################################
0:	test: 0.7052101	best: 0.7052101 (0)	total: 41.8ms	remaining: 1m 44s
500:	t

### 5-3. XGBoost

In [23]:
max_params_xgb = {
	'colsample_bytree': 0.83100305182246,
	'gamma': 8.0503356372611,
	'max_depth': 8,
	'min_child_weight': 6.617094996274681,
	'reg_alpha': 7.3168709375076535,
	'reg_lambda': 1.2463377314671225,
	'scale_pos_weight': 1.5214260635820143,
	'subsample': 0.879872640465336,
	'objective': 'binary:logistic',
	'learning_rate': 0.02,
	'random_state': 1991
 }

In [24]:
import xgboost as xgb

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds_xgb = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds_xgb = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # XGBoost 전용 데이터셋 생성 
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest = xgb.DMatrix(X_test)

    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=max_params_xgb, 
                          dtrain=dtrain,
                          num_boost_round=2500,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          custom_metric=xgb_roc_auc, 
                          callbacks=[xgb.callback.EarlyStopping(rounds=200)],
                          verbose_eval=0)

    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration

    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds_xgb += xgb_model.predict(dtest,
                                            iteration_range=(0, best_iter))/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds_xgb[valid_idx] += xgb_model.predict(dvalid, 
                                                      iteration_range=(0, best_iter))
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(dvalid.get_label(), oof_val_preds_xgb[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
폴드 1 roc-auc : 0.7378778297062083

######################################## 폴드 2 / 폴드 10 ########################################
폴드 2 roc-auc : 0.7370134357672872

######################################## 폴드 3 / 폴드 10 ########################################
폴드 3 roc-auc : 0.7323590117639232

######################################## 폴드 4 / 폴드 10 ########################################
폴드 4 roc-auc : 0.7376489832831726

######################################## 폴드 5 / 폴드 10 ########################################
폴드 5 roc-auc : 0.7408424454318432

######################################## 폴드 6 / 폴드 10 ########################################
폴드 6 roc-auc : 0.7348122179022862

######################################## 폴드 7 / 폴드 10 ########################################
폴드 7 roc-auc : 0.7434894580436002

######################################## 폴드 8 / 폴드 10 ####################################

## 6. 최종 결과

In [19]:
print('LightGBM OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, oof_val_preds_lgb))

LightGBM OOF 검증 데이터 ROC-AUC : 0.7404986919647051


In [22]:
print('CatBoost OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, oof_val_preds_catboost))

CatBoost OOF 검증 데이터 ROC-AUC : 0.739691181269408


In [25]:
print('XGBoost OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, oof_val_preds_xgb))

XGBoost OOF 검증 데이터 ROC-AUC : 0.7386423346574369


In [43]:
ensemble_preds = oof_val_preds_lgb * 0.8 + oof_val_preds_catboost * 0.2
# 앙상블한 예측값으로 ROC-AUC 확인
ensemble_roc_auc = roc_auc_score(y, ensemble_preds)
print(f"앙상블 OOF 검증 데이터 ROC-AUC: {ensemble_roc_auc}")

앙상블 OOF 검증 데이터 ROC-AUC: 0.7405805988289571


In [42]:
ensemble_preds = oof_val_preds_lgb * 0.75 + oof_val_preds_catboost * 0.25

# 앙상블한 예측값으로 ROC-AUC 확인
ensemble_roc_auc = roc_auc_score(y, ensemble_preds)
print(f"앙상블 OOF 검증 데이터 ROC-AUC: {ensemble_roc_auc}")

앙상블 OOF 검증 데이터 ROC-AUC: 0.7405836119014515


In [45]:
# LightGBM과 XGBoost 앙상블 예측 (가중 평균)
ensemble_preds = oof_val_preds_lgb * 0.75 + oof_val_preds_catboost * 0.15 + oof_val_preds_xgb * 0.05

# 앙상블한 예측값으로 ROC-AUC 확인
ensemble_roc_auc = roc_auc_score(y, ensemble_preds)
print(f"앙상블 OOF 검증 데이터 ROC-AUC: {ensemble_roc_auc}")

앙상블 OOF 검증 데이터 ROC-AUC: 0.7405630070728243


In [34]:
ensemble_test_preds = oof_test_preds_lgb * 0.75 + oof_test_preds_catboost * 0.25
submission['probability'] = ensemble_test_preds
submission.to_csv('ensemble_lgbm_cat_submission.csv')